In [345]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import seaborn as sns
import matplotlib.pyplot as plt

import string
import ftfy

import warnings
warnings.filterwarnings("ignore")

In [346]:
df_ratings_scrapped = pd.read_csv('Datasets/df_ratings.csv')
df_ratings_scrapped.shape

(52944, 5)

In [347]:
# deleting albums with less than 30 votes
df_ratings_scrapped = df_ratings_scrapped[df_ratings_scrapped['votes']>30]
df_ratings_scrapped.reset_index(drop=True, inplace=True)
df_ratings_scrapped.shape

(20665, 5)

**Checking null values**

In [348]:
df_ratings_scrapped[df_ratings_scrapped['album'].isna()]

,album_id,artist,album,rating,votes
15417,35979,Meshuggah,NaN,3.78,314


In [349]:
# replace the null values for the actual name of the album
df_ratings_scrapped['album'] = np.where(df_ratings_scrapped['album_id']==35979, 'None' , df_ratings_scrapped['album'])

**Applying ``ftfy.fix_text``**

In [ ]:
album = ftfy.fix_text('Let It Beâ¦ Naked')
album

'Let It Be… Naked'

In [ ]:
df_ratings_scrapped['album'] = df_ratings_scrapped['album'].apply(ftfy.fix_text)

**Defining a function to detect strange characters**

In [350]:
letters = list(string.ascii_letters)
other_characters = ['-', '.', '+', ' ', '/']
valid_characters = (letters) + list(range(0,10)) + other_characters
len(valid_characters)

67

In [371]:
def valid_title(x):
    album = ''.join(x.split(' '))
    numbers = list(str(i) for i in range(10))
    letters = list(string.ascii_letters)
    other_characters = ['-', '.', '+', '/', '&', ',', "'", '¡', '!', '¿', '?', ':', '(', ')', '=', '"', '[', ']', '#', '%', '_', '$', '*', '@', '…', 'é']
    valid_characters = (letters) + numbers + other_characters

    for char in album:
        if char not in valid_characters:
            return False
    return True

In [372]:
valid_title('Let It Be… Naked')

True

**Applying the function**

In [373]:
df_ratings_scrapped['valid_title'] = df_ratings_scrapped['album'].apply(valid_title)
df_ratings_scrapped['valid_title'].value_counts()

valid_title
True     20515
False      150
Name: count, dtype: int64

In [378]:
df_ratings_scrapped[df_ratings_scrapped['valid_title']==False].sort_values('album_id').head(30)

,album_id,artist,album,rating,votes,valid_title
95,172,Tool,Ænima,4.43,6499,False
337,474,Fantomas,Fantômas,3.53,251,False
632,855,Jeff Buckley,Live à L'Olympia,3.96,70,False
750,1013,Leftover Crack,Fuсk World Trade,3.81,280,False
933,1240,Sigur Ros,Agætis byrjun,4.37,2791,False
1040,1388,The Mothers of Invention,Fillmore East – June 1971,3.82,71,False
1126,1512,Godspeed You! Black Emperor,F♯ A♯ ∞,4.48,3962,False
1200,1621,Finntroll,Nattfödd,3.94,295,False
1640,2455,Fantomas,Delìrium Còrdia,3.56,284,False
1878,2812,Atomship,The Crash of `47,4.13,161,False
